In [15]:
import requests
import json
import pandas as pd

# Your CoinMarketCap API key
API_KEY = ' '  # Replace with your actual API key

# Base URL for the CoinMarketCap API
BASE_URL = 'https://pro-api.coinmarketcap.com/v1'

# Endpoint to get the latest listings of cryptocurrencies
LISTINGS_ENDPOINT = '/cryptocurrency/listings/latest'

# Parameters for the request
parameters = {
    'start': '1',
    'limit': '100',  # Get the top 100 cryptocurrencies
    'convert': 'USD'  # Convert prices to USD
}

headers = {
    'Accepts': 'application/json',
    'X-CMC_PRO_API_KEY': API_KEY,
}

try:
    # Make the API request
    response = requests.get(BASE_URL + LISTINGS_ENDPOINT, headers=headers, params=parameters)
    response.raise_for_status()  # Raise an exception for bad status codes

    # Parse the JSON response
    data = response.json()['data']

    # Convert the JSON data to a Pandas DataFrame for easier handling
    df = pd.DataFrame(data)

    # Function to extract and format USD quote data
    def extract_and_format_usd_quote(quote):
        if isinstance(quote, dict) and 'USD' in quote:
            usd_data = quote['USD']
            # Format the price, handling potential errors
            try:
                price = float(usd_data.get('price', 0)) #important
                price = round(price, 2)  # Format to 2 decimal places
            except (ValueError, TypeError):
                price = 0.0  # Or some other default value, handle error

            volume_24h = usd_data.get('volume_24h')
            volume_change_24h = round(usd_data.get('volume_change_24h', 0), 1)

            # Round the percentage change columns to one decimal place
            percent_change_1h = round(usd_data.get('percent_change_1h', 0), 1)
            percent_change_24h = round(usd_data.get('percent_change_24h', 0), 1)
            percent_change_7d = round(usd_data.get('percent_change_7d', 0), 1)
            percent_change_30d = round(usd_data.get('percent_change_30d', 0), 1)
            percent_change_60d = round(usd_data.get('percent_change_60d', 0), 1)
            percent_change_90d = round(usd_data.get('percent_change_90d', 0), 1)
            market_cap_dominance = round(usd_data.get('market_cap_dominance', 0), 1)

            market_cap = usd_data.get('market_cap')
            last_updated = usd_data.get('last_updated')
            return {
                'price': price,
                'volume_24h': volume_24h,
                'volume_change_24h': volume_change_24h,
                'percent_change_1h': percent_change_1h,
                'percent_change_24h': percent_change_24h,
                'percent_change_7d': percent_change_7d,
                'percent_change_30d': percent_change_30d,
                'percent_change_60d': percent_change_60d,
                'percent_change_90d': percent_change_90d,
                'market_cap': market_cap,
                'market_cap_dominance': market_cap_dominance,
                'last_updated': last_updated
            }
        return {}

    # Apply the function to the 'quote' column to extract and format data
    df_usd_quote = df['quote'].apply(extract_and_format_usd_quote).apply(pd.Series)

    # Concatenate the desired columns with the original DataFrame
    df = pd.concat([df[['id', 'name', 'symbol']], df_usd_quote], axis=1)

    # Select the desired columns
    df = df[['name', 'symbol', 'price', 'volume_24h', 'volume_change_24h',
               'percent_change_24h', 'percent_change_7d', 'percent_change_30d',
               'percent_change_60d', 'percent_change_90d', 'market_cap',
               'market_cap_dominance', 'last_updated']]

    # Convert 'last_updated' to datetime and format it
    df['date'] = pd.to_datetime(df['last_updated']).dt.strftime('%Y-%m-%d')

    # Drop the original 'last_updated' column
    df = df.drop(columns=['last_updated'])

    # Display the first few rows of the cleaned DataFrame
    print("\nCleaned and Formatted DataFrame:")
    print(df.head())

    # Save the cleaned DataFrame to a CSV file
    df.to_csv('cryptocurrency_data_cleaned.csv', index=False)
    print("\nCleaned data saved to cryptocurrency_data_cleaned.csv")

except requests.exceptions.RequestException as e:
    print(f"API request error: {e}")
except json.JSONDecodeError as e:
    print(f"JSON decoding error: {e}")
except KeyError as e:
    print(f"Key error in JSON response: {e}")
except Exception as e:
    print(f"An unexpected error occurred: {e}")


Cleaned and Formatted DataFrame:
          name symbol     price    volume_24h  volume_change_24h  \
0      Bitcoin    BTC  87109.69  4.173890e+10              180.0   
1     Ethereum    ETH   1576.58  1.596131e+10               92.5   
2  Tether USDt   USDT      1.00  7.181927e+10               95.4   
3          XRP    XRP      2.07  2.733726e+09               59.3   
4          BNB    BNB    596.71  1.432390e+09               10.2   

   percent_change_24h  percent_change_7d  percent_change_30d  \
0                 2.7                3.1                 3.8   
1                -0.3               -2.7               -20.6   
2                 0.0                0.0                 0.0   
3                 0.3               -2.8               -12.4   
4                 1.1                1.8                -4.8   

   percent_change_60d  percent_change_90d    market_cap  market_cap_dominance  \
0               -11.3               -17.9  1.729514e+12                  63.5   
1         

In [16]:
df.head()

,name,symbol,price,volume_24h,volume_change_24h,percent_change_24h,percent_change_7d,percent_change_30d,percent_change_60d,percent_change_90d,market_cap,market_cap_dominance,date
0,Bitcoin,BTC,87109.69,4.173890e+10,180.0,2.7,3.1,3.8,-11.3,-17.9,1.729514e+12,63.5,2025-04-21
1,Ethereum,ETH,1576.58,1.596131e+10,92.5,-0.3,-2.7,-20.6,-42.2,-52.8,1.903057e+11,7.0,2025-04-21
2,Tether USDt,USDT,1.00,7.181927e+10,95.4,0.0,0.0,0.0,-0.0,-0.0,1.446402e+11,5.3,2025-04-21
3,XRP,XRP,2.07,2.733726e+09,59.3,0.3,-2.8,-12.4,-22.8,-34.8,1.210884e+11,4.4,2025-04-21
4,BNB,BNB,596.71,1.432390e+09,10.2,1.1,1.8,-4.8,-9.2,-14.0,8.407056e+10,3.1,2025-04-21


In [25]:
import ccxt
import pandas as pd
from datetime import datetime

# Initialize the exchange (using Binance as example)
exchange = ccxt.binance({
    'enableRateLimit': True,  # Required by CCXT to avoid rate limits
})

# Define parameters
symbol = 'BTC/USDT'
timeframe = '1d'
since_date = '2024-08-19'
output_filename = 'btc_daily_data.csv'

# Convert since_date to timestamp in milliseconds
since_timestamp = exchange.parse8601(since_date + 'T00:00:00Z')

try:
    print(f"Fetching daily BTC/USDT data since {since_date}...")
    
    # Fetch OHLCV data
    ohlcv = exchange.fetch_ohlcv(symbol, timeframe, since_timestamp)
    
    # Convert to pandas DataFrame
    df = pd.DataFrame(ohlcv, columns=['timestamp', 'open', 'high', 'low', 'close', 'volume'])
    
    # Convert timestamp to datetime
    df['date'] = pd.to_datetime(df['timestamp'], unit='ms')
    
    # Reorder columns
    df = df[['date', 'open', 'high', 'low', 'close', 'volume']]
    
    # Save to CSV
    df.to_csv(output_filename, index=False)
    print(f"Data successfully saved to {output_filename}")
    print(f"Total records fetched: {len(df)}")
    
except Exception as e:
    print(f"An error occurred: {str(e)}")

Fetching daily BTC/USDT data since 2024-08-19...
Data successfully saved to btc_daily_data.csv
Total records fetched: 247


In [6]:
import requests
import json
import pandas as pd
import ccxt
from google.cloud import bigquery
from google.oauth2 import service_account
from datetime import datetime

# Configuration
COINMARKETCAP_API_KEY = '76f47bca-8f25-4131-a16e-31078d13d071'
SERVICE_ACCOUNT_PATH = r"C:\Users\drewr\OneDrive\Escritorio\Google Cloud Training\Crypto Dashboard\Crypto API Data\key-btc-investment-1.json"
PROJECT_ID = 'btc-investment-1'
DATASET_ID = 'Crypto_Dataset_Bucket'

# Initialize BigQuery client
credentials = service_account.Credentials.from_service_account_file(SERVICE_ACCOUNT_PATH)
bq_client = bigquery.Client(project=PROJECT_ID, credentials=credentials)

def create_dataset():
    """Creates the BigQuery dataset if it doesn't exist"""
    dataset_ref = bigquery.DatasetReference(PROJECT_ID, DATASET_ID)
    dataset = bigquery.Dataset(dataset_ref)
    dataset.location = "US"
    
    try:
        dataset = bq_client.create_dataset(dataset, exists_ok=True)
        print(f"Dataset {DATASET_ID} created or already exists")
    except Exception as e:
        print(f"Error creating dataset: {str(e)}")

def upload_to_bigquery(df, table_name):
    """Uploads data to BigQuery with WRITE_TRUNCATE to replace existing data"""
    table_ref = f"{PROJECT_ID}.{DATASET_ID}.{table_name}"
    
    job_config = bigquery.LoadJobConfig(
        autodetect=True,
        write_disposition=bigquery.WriteDisposition.WRITE_TRUNCATE  # This overwrites the table
    )
    
    try:
        job = bq_client.load_table_from_dataframe(df, table_ref, job_config=job_config)
        job.result()
        print(f"Successfully replaced all data in {table_ref} with {len(df)} rows")
    except Exception as e:
        print(f"Error uploading to BigQuery: {str(e)}")

def get_coinmarketcap_data():
    BASE_URL = 'https://pro-api.coinmarketcap.com/v1'
    LISTINGS_ENDPOINT = '/cryptocurrency/listings/latest'
    
    parameters = {
        'start': '1',
        'limit': '100',
        'convert': 'USD'
    }
    
    headers = {
        'Accepts': 'application/json',
        'X-CMC_PRO_API_KEY': COINMARKETCAP_API_KEY,
    }
    
    try:
        response = requests.get(BASE_URL + LISTINGS_ENDPOINT, headers=headers, params=parameters)
        response.raise_for_status()
        data = response.json()['data']
        df = pd.DataFrame(data)

        def extract_and_format_usd_quote(quote):
            if isinstance(quote, dict) and 'USD' in quote:
                usd_data = quote['USD']
                try:
                    price = round(float(usd_data.get('price', 0)), 2)
                except (ValueError, TypeError):
                    price = 0.0

                return {
                    'price': price,
                    'volume_24h': usd_data.get('volume_24h'),
                    'volume_change_24h': round(usd_data.get('volume_change_24h', 0), 1),
                    'percent_change_1h': round(usd_data.get('percent_change_1h', 0), 1),
                    'percent_change_24h': round(usd_data.get('percent_change_24h', 0), 1),
                    'percent_change_7d': round(usd_data.get('percent_change_7d', 0), 1),
                    'percent_change_30d': round(usd_data.get('percent_change_30d', 0), 1),
                    'percent_change_60d': round(usd_data.get('percent_change_60d', 0), 1),
                    'percent_change_90d': round(usd_data.get('percent_change_90d', 0), 1),
                    'market_cap': usd_data.get('market_cap'),
                    'market_cap_dominance': round(usd_data.get('market_cap_dominance', 0), 1),
                    'last_updated': usd_data.get('last_updated')
                }
            return {}

        df_usd_quote = df['quote'].apply(extract_and_format_usd_quote).apply(pd.Series)
        df = pd.concat([df[['id', 'name', 'symbol']], df_usd_quote], axis=1)
        
        df = df[['name', 'symbol', 'price', 'volume_24h', 'volume_change_24h',
                 'percent_change_24h', 'percent_change_7d', 'percent_change_30d',
                 'percent_change_60d', 'percent_change_90d', 'market_cap',
                 'market_cap_dominance', 'last_updated']]
        
        df['date'] = pd.to_datetime(df['last_updated']).dt.strftime('%Y-%m-%d')
        df = df.drop(columns=['last_updated'])
        
        print("\nCleaned and Formatted DataFrame:")
        print(df.head())
        
        upload_to_bigquery(df, 'API')
        
    except requests.exceptions.RequestException as e:
        print(f"API request error: {e}")
    except json.JSONDecodeError as e:
        print(f"JSON decoding error: {e}")
    except KeyError as e:
        print(f"Key error in JSON response: {e}")
    except Exception as e:
        print(f"An unexpected error occurred: {e}")

def get_ccxt_data():
    exchange = ccxt.binance({'enableRateLimit': True})
    symbol = 'BTC/USDT'
    timeframe = '1d'
    since_date = '2024-08-19'
    
    try:
        since_timestamp = exchange.parse8601(since_date + 'T00:00:00Z')
        print(f"Fetching daily BTC/USDT data since {since_date}...")
        
        ohlcv = exchange.fetch_ohlcv(symbol, timeframe, since_timestamp)
        df = pd.DataFrame(ohlcv, columns=['timestamp', 'open', 'high', 'low', 'close', 'volume'])
        df['date'] = pd.to_datetime(df['timestamp'], unit='ms')
        df = df[['date', 'open', 'high', 'low', 'close', 'volume']]
        
        print(f"Total records fetched: {len(df)}")
        upload_to_bigquery(df, 'CCXT')
        
    except Exception as e:
        print(f"An error occurred: {str(e)}")

if __name__ == "__main__":
    create_dataset()
    get_coinmarketcap_data()
    get_ccxt_data()

Dataset Crypto_Dataset_Bucket created or already exists

Cleaned and Formatted DataFrame:
          name symbol     price    volume_24h  volume_change_24h  \
0      Bitcoin    BTC  91468.91  4.607258e+10               10.6   
1     Ethereum    ETH   1700.96  1.992568e+10               24.4   
2  Tether USDt   USDT      1.00  8.250662e+10               15.0   
3          XRP    XRP      2.15  3.157191e+09               13.8   
4          BNB    BNB    608.47  1.620065e+09               13.1   

   percent_change_24h  percent_change_7d  percent_change_30d  \
0                 4.8                8.9                 7.4   
1                 7.9                5.6               -14.9   
2                 0.1                0.1                 0.1   
3                 3.3                1.6               -10.4   
4                 2.0                4.3                -2.3   

   percent_change_60d  percent_change_90d    market_cap  market_cap_dominance  \
0                -4.1              

c:\Users\drewr\AppData\Local\Programs\Python\Python313\Lib\site-packages\google\cloud\bigquery\_pandas_helpers.py:489: FutureWarning: Loading pandas DataFrame into BigQuery will require pandas-gbq package version 0.26.1 or greater in the future. Tried to import pandas-gbq and got: No module named 'pandas_gbq'
  warnings.warn(


Successfully replaced all data in btc-investment-1.Crypto_Dataset_Bucket.API with 100 rows
Fetching daily BTC/USDT data since 2024-08-19...
Total records fetched: 247


c:\Users\drewr\AppData\Local\Programs\Python\Python313\Lib\site-packages\google\cloud\bigquery\_pandas_helpers.py:489: FutureWarning: Loading pandas DataFrame into BigQuery will require pandas-gbq package version 0.26.1 or greater in the future. Tried to import pandas-gbq and got: No module named 'pandas_gbq'
  warnings.warn(


Successfully replaced all data in btc-investment-1.Crypto_Dataset_Bucket.CCXT with 247 rows
